## CAI - Rebalancing data for September 2023 

## Configuring packages

In [18]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
import datetime
import time
sys.path.append('../')

from web3 import Web3
from abis import index_anatomy


infura_url = decouple.config("AVALANCHE_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

import db_funcs

In [19]:
from pycoingecko import CoinGeckoAPI
key = decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key= key)

In [20]:
import plotly.express as px
import plotly.graph_objects as go

In [21]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables and settings

In [22]:
db_key_date = '2023-09-01'
db_liquidity_table = 'cai_liquidities'
db_benchmark_table = 'cai_benchmark_data'
min_weight = 0.005
max_weight = 0.5
supply_threshold = 0.3
liquidity_consistency = 90 # In days
avax_to_savax_split = 0.6
liveness_threshold = 30*6 # In days

In [23]:
liq = 2*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.09

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe, Platypus, Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [24]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### CoinGecko

##### Tokens addresses

In [25]:
coins_list = cg.get_coins_list(include_platform=True)
coins = {}
for coin in coins_list:
    if 'avalanche' in coin['platforms']:
        coin['addr'] = coin['platforms']['avalanche']
        coin['symbol'] = normalize_symb(coin['symbol'])
        coins[coin['id']] = coin

# Adding AVAX
coins['avalanche-2'] = {'id': 'avalanche-2','symbol':normalize_symb('avax'), 'name':'Avalanche','addr':'0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7',}


##### Market data

In [26]:
cat_id = "avalanche-ecosystem"
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id)
for i in range(2, 30):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id, page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)


#print(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'))

cg_tokens_data = pd.concat([cg_tokens_data,pd.DataFrame(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'))],ignore_index=True)
cg_tokens_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.000000,8.289634e+10,3.0,8.289634e+10,6.580590e+09,1.001000,...,8.290017e+10,NaN,1.320000,-24.43044,2018-07-24T00:00:00.000Z,0.572521,74.64115,2015-03-02T00:00:00.000Z,None,2023-09-03T02:15:00.950Z
1,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,213.530000,3.286530e+10,4.0,4.272212e+10,2.381645e+08,215.270000,...,1.538562e+08,2.000000e+08,686.310000,-68.87328,2021-05-10T07:24:17.097Z,0.039818,536406.94205,2017-10-19T00:00:00.000Z,None,2023-09-03T02:19:16.808Z
2,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.000000,2.614963e+10,6.0,2.614962e+10,1.559181e+09,1.002000,...,2.615646e+10,NaN,1.170000,-14.74120,2019-05-08T00:40:28.300Z,0.877647,13.92230,2023-03-11T08:02:13.981Z,None,2023-09-03T02:19:26.167Z
3,wrapped-bitcoin,wbtc,Wrapped Bitcoin,https://assets.coingecko.com/coins/images/7598...,25869.000000,4.209571e+09,17.0,4.209571e+09,2.348646e+07,25944.000000,...,1.628749e+05,1.628749e+05,70643.000000,-63.40249,2021-11-10T14:40:19.650Z,3139.170000,723.58404,2019-04-02T00:00:00.000Z,None,2023-09-03T02:19:26.054Z
4,dai,dai,Dai,https://assets.coingecko.com/coins/images/9956...,0.999939,3.874925e+09,18.0,3.874925e+09,5.119232e+07,1.001000,...,3.875441e+09,NaN,1.220000,-17.96389,2020-03-13T03:02:50.373Z,0.881960,13.37967,2023-03-11T07:50:50.514Z,None,2023-09-03T02:15:00.781Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,sol-wormhole,sol,SOL (Wormhole),https://assets.coingecko.com/coins/images/2287...,19.500000,0.000000e+00,NaN,NaN,4.336100e+04,19.840000,...,NaN,NaN,141.630000,-86.23316,2022-04-02T14:00:59.236Z,8.230000,136.81447,2022-12-29T20:51:26.429Z,None,2023-09-03T02:19:22.512Z
325,space-token-bsc,space,Space Token BSC,https://assets.coingecko.com/coins/images/2067...,0.019813,0.000000e+00,NaN,6.886835e+06,6.741170e+03,0.019994,...,3.494807e+08,3.500000e+08,0.094492,-79.14866,2021-12-01T01:09:10.465Z,0.009770,101.67476,2022-05-12T05:56:03.100Z,None,2023-09-03T02:19:20.018Z
326,brz,brz,Brazilian Digital,https://assets.coingecko.com/coins/images/8472...,0.224408,0.000000e+00,NaN,2.020421e+06,5.956060e+05,0.224978,...,9.000000e+06,NaN,6.540000,-96.56711,2023-05-18T12:08:35.386Z,0.005509,3973.98467,2023-06-09T04:54:52.829Z,None,2023-09-03T02:19:22.172Z
327,brokkr,bro,Brokkr,https://assets.coingecko.com/coins/images/2781...,0.000956,0.000000e+00,NaN,9.559870e+05,2.098900e+02,0.000958,...,1.000000e+09,1.000000e+09,0.017118,-94.41525,2022-10-22T12:04:20.721Z,0.000867,10.21159,2023-08-22T16:35:12.404Z,None,2023-09-02T15:30:09.181Z


In [27]:
tokens_market_data = cg_tokens_data.set_index('id')[
    ['name', 'symbol', 'market_cap', 'market_cap_rank', 'current_price', 'total_supply', 'circulating_supply']
]
tokens_market_data['symbol'] = tokens_market_data['symbol'].apply(normalize_symb)

In [28]:
tokens_market_data = tokens_market_data[tokens_market_data['market_cap'] > 0]
tokens_market_data = tokens_market_data[~tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.join(pd.DataFrame.from_dict(coins, orient='index')['addr'])
tokens_market_data = tokens_market_data[~tokens_market_data['addr'].isna()]


In [29]:
# Stable coins to remove from tokens list
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')

#Adjusting Pangolin's max supply to reflect tokenomic changes here: https://app.pangolin.exchange/#/vote/10
tokens_market_data.loc[['pangolin'],["total_supply"]] = 230000000.0

tokens_market_data =  tokens_market_data[~tokens_market_data.index.isin(stablecoins.index)]
tokens_market_data = tokens_market_data.sort_values('market_cap', ascending=False)

tokens_market_data

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
wrapped-bitcoin,Wrapped Bitcoin,WBTC,4.209571e+09,17.0,25869.000000,1.628749e+05,1.628749e+05,0x50b7545627a5162f82a992c33b87adc75187b218
avalanche-2,Avalanche,AVAX,3.502275e+09,21.0,9.910000,4.309339e+08,3.535578e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
uniswap,Uniswap,UNI,3.292656e+09,23.0,4.370000,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
chainlink,Chainlink,LINK,3.214057e+09,24.0,5.980000,1.000000e+09,5.381000e+08,0x5947bb275c521040051d82396192181b413227a3
maker,Maker,MKR,1.021032e+09,43.0,1135.100000,9.776310e+05,9.013109e+05,0x88128fd4b259552a9a1d457f435a6527aab72d42
...,...,...,...,...,...,...,...,...
hakuswap,HakuSwap,HAKU,7.055590e+03,3510.0,0.000168,8.237168e+07,4.201604e+07,0x695fa794d59106cebd40ab5f5ca19f458c723829
defrost-finance,Defrost Finance,MELT,6.291050e+03,3524.0,0.000414,1.000000e+08,1.519648e+07,0x47eb6f7525c1aa999fbc9ee92715f5231eb1241d
echidna,Echidna,ECD,5.934760e+03,3528.0,0.000516,3.382947e+08,1.149388e+07,0xeb8343d5284caec921f035207ca94db6baaacbcd


### Token's supply check 

In [30]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > supply_threshold
sp_passed = tokens_market_data[sp_passed_index]

### Retrieve historical data & token age check

In [31]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id_, row in sp_passed.iterrows():
    symbol = row['symbol']
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', id_])
    df_prices = df_prices[df_prices[id_] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < liveness_threshold:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {liveness_threshold} days')
        exclude_list.append(id_)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id_])
    df_mcaps = df_mcaps[df_mcaps[id_] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < liveness_threshold:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {liveness_threshold} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Note: NXRA, marketcap data available only for 167 < 180 days
Excluding MELD, prices data available only for 118 < 180 days
Excluding USP, prices data available only for 13 < 180 days
Excluding 3ULL, prices data available only for 86 < 180 days
Note: EIV, marketcap data available only for 18 < 180 days
Excluding ERN, prices data available only for 147 < 180 days
Note: EUROE, marketcap data available only for 74 < 180 days
Note: GMD, marketcap data available only for 96 < 180 days
Excluding CINU, prices data available only for 174 < 180 days
Excluding GRAIN, prices data available only for 143 < 180 days
Note: PLN, marketcap data available only for 95 < 180 days


### Non quantative checks

In [32]:
manual_exclusions = [
    {
        'gecko_id': 'spell-token',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'stacktical',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'zookeeper',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pendle',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'everrise',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'wrapped-bitcoin',
        'reason' : 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'bitcoin-avalanche-bridged-btc-b',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'gmx',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'woo-network',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'euro-coin',
        'reason': 'Stable coin'
    }
]
for token in manual_exclusions:
    exclude_list.append(token['gecko_id'])

In [33]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list')
passed_q_and_non_q_checks.drop(passed_q_and_non_q_checks[passed_q_and_non_q_checks['market_cap'] < 1e6].index)

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,3.502275e+09,21.0,9.910000e+00,4.309339e+08,3.535578e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
uniswap,Uniswap,UNI,3.292656e+09,23.0,4.370000e+00,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
chainlink,Chainlink,LINK,3.214057e+09,24.0,5.980000e+00,1.000000e+09,5.381000e+08,0x5947bb275c521040051d82396192181b413227a3
maker,Maker,MKR,1.021032e+09,43.0,1.135100e+03,9.776310e+05,9.013109e+05,0x88128fd4b259552a9a1d457f435a6527aab72d42
aave,Aave,AAVE,7.964972e+08,47.0,5.486000e+01,1.600000e+07,1.452862e+07,0x63a72806098bd3d9520cc43356dd78afe5d386d9
the-graph,The Graph,GRT,7.923894e+08,48.0,8.634500e-02,1.000000e+10,9.175013e+09,0x8a0cac13c7da965a312f08ea4229c37869e85cb9
havven,Synthetix Network,SNX,6.386693e+08,57.0,1.980000e+00,3.235067e+08,3.227828e+08,0xbec243c995409e6520d7c41e404da5deba4b209b
frax-share,Frax Share,FXS,4.158246e+08,87.0,5.670000e+00,9.968150e+07,7.337133e+07,0x214db107654ff987ad859f34125307783fc8e387
compound-governance-token,Compound,COMP,2.766364e+08,114.0,4.033000e+01,1.000000e+07,6.856086e+06,0xc3048e19e76cb9a3aa9d77d8c03c29fc906e2437


### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [36]:
url_0x = "https://avalanche.api.0x.org/swap/v1/quote"
header =  {'0x-api-key': decouple.config("ZEROEX_KEY")}

In [37]:
decimals = 18
slippages = []
for id, data in passed_q_and_non_q_checks.iterrows():
    try:
        query = {
            'buyToken': data['addr'],
            'sellToken': 'AVAX',
            'sellAmount': int(trade_value_tiny / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query,headers=header)
        swap = resp.json()
        time.sleep(1)
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals)
        resp = requests.get(url_0x, params=query,headers=header)
        swap = resp.json()
        time.sleep(1)
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1
        slippages.append({
            'id': id,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
    except KeyError:
        print(id)
        continue
    

havven
balancer
axelar
cartesi
ankreth
allianceblock-nexera
idia
realfevr
oath
euroe-stablecoin
build
spherium
soul-swap
shack
bios


In [38]:
slippages_df = pd.DataFrame(slippages).sort_values('slippage',ascending=False).reset_index(drop=True)
slippages_df.head(20)

,id,spot_price,delivery_price,slippage
0,avalanche-2,1.000000,1.000000,0.000000
1,benqi-liquid-staked-avax,0.907215,0.907193,-0.000024
2,joe,37.731427,36.560857,-0.031024
3,colony-avalanche-index,0.131312,0.119449,-0.090343
4,benqi,1959.708556,1775.550580,-0.093972
5,pangolin,482.390568,395.089132,-0.180977
6,avalaunch,60.092434,48.605701,-0.191151
7,chainlink,1.654738,1.265096,-0.235471
8,yield-yak,0.035700,0.020195,-0.434315
9,aave,0.179149,0.093552,-0.477797


In [39]:
# Writing to database
slippages_to_save = slippages_df[['id','slippage']].head(20)
asset_columns = db_funcs.convert_to_sql_strings(list(slippages_to_save['id']))
slippage_values = list(slippages_to_save['slippage'])
db_funcs.insert_values(db_key_date,asset_columns,slippage_values,db_liquidity_table)


In [40]:
def check_avg_liquidity(date_delta):
    start_date = str(datetime.date.fromisoformat(db_key_date) - datetime.timedelta(days=date_delta))
    dataframe = pd.read_sql(f'Select * from {db_liquidity_table} where date >= ? ',db_funcs.create_connection(db_funcs.db),index_col='date',parse_dates=['date'],params=[start_date])
    dataframe = dataframe.ewm(span=3).mean().iloc[-1]
    dataframe = dataframe[dataframe > max_slippage]
    dataframe.index = db_funcs.convert_from_sql_strings(list(dataframe.index))
    return dataframe, list(dataframe.index)

avg_liq_dataframe ,include_list = check_avg_liquidity(liquidity_consistency)
avg_liq_dataframe


benqi-liquid-staked-avax   -0.000157
avalanche-2                 0.000000
joe                        -0.018925
Name: 2023-09-01 00:00:00, dtype: float64

In [41]:
qualified_assets = passed_q_and_non_q_checks.query('index in @include_list')
qualified_assets


,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,3.502275e+09,21.0,9.910000,4.309339e+08,3.535578e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
joe,JOE,JOE,8.912952e+07,250.0,0.262307,4.764376e+08,3.414594e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,6.991031e+07,303.0,10.910000,6.409174e+06,6.409174e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be


### Marketcap ranking & filtering

In [42]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens. 
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [43]:
# exclude savax from weighting since it should be a part of avax weight
if 'benqi-liquid-staked-avax' in chosen_tokens.index:
    chosen_tokens_ex_savax = chosen_tokens.drop('benqi-liquid-staked-avax')
    weights = chosen_tokens_ex_savax['market_cap'].div(chosen_tokens_ex_savax['market_cap'].sum())
else:
    weights = chosen_tokens['market_cap'].div(chosen_tokens['market_cap'].sum())



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [44]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum()
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum()).mul(c)
    
    return w

In [45]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum())
    
    return w

In [46]:
adjusted_weights = adjust_weights(weights, max_weight=max_weight) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=min_weight)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=max_weight)

### Split AVAX 

In [47]:
if 'avalanche-2' and 'benqi-liquid-staked-avax' in chosen_tokens.index:
    
    adjusted_weights['benqi-liquid-staked-avax'] = adjusted_weights['avalanche-2'] * avax_to_savax_split
    adjusted_weights['avalanche-2'] = adjusted_weights['avalanche-2'] * (1 - avax_to_savax_split)

adjusted_weights =  adjusted_weights.sort_values(ascending = False)


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [48]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<128]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [49]:
converted_weights = convert_weights(adjusted_weights)

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_35398/3538882253.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_35398/3538882253.py:8: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`



## Summary table

In [50]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['symbol'] = chosen_tokens['symbol']
cai['market_cap'] = chosen_tokens['market_cap']
cai['price'] = chosen_tokens['current_price']
cai['weight'] = adjusted_weights
cai['weight_converted'] = converted_weights
cai['address'] = chosen_tokens['addr']

cai = cai[cai['weight'] > 0]
cai = cai.sort_values("market_cap", ascending=False)
cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,3.502275e+09,9.910000,0.2,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
joe,JOE,JOE,8.912952e+07,0.262307,0.5,128,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,6.991031e+07,10.910000,0.3,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be


In [51]:
# Add assets and weights to database
db_assets = db_funcs.convert_to_sql_strings(list(cai.index))
db_weights = list(cai['weight'])
db_funcs.insert_values(db_key_date,db_assets,db_weights,db_benchmark_table)

In [52]:
#Querying current contract to ascertain which assets will be dropped

cai_token_address = '0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0'
ia_contract = web3_object.eth.contract(address=cai_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()


for address in addresses:
    address = address.lower()
    if address not in list(cai['address']):
        data = tokens_market_data[tokens_market_data['addr']== address]
        cai.loc[data.index[0]] = [data['name'][0],data['symbol'][0],data['market_cap'][0],data['current_price'][0],0,0,address]

cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,3.502275e+09,9.910000,0.2,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
joe,JOE,JOE,8.912952e+07,0.262307,0.5,128,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,6.991031e+07,10.910000,0.3,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be


In [53]:
zipped_assets = list(zip(list(cai['address']),list(cai['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = ''
for i in sorted_assets:
    asset_string += f'{i[0]},'
asset_string = asset_string[:-1]
print(asset_string)
print([x[1] for x in sorted_assets])

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
[76, 128, 51]
